### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2018 ###

# Hobby Farm Production Optimization #

#### Joel Atkins (jratkins@wisc.edu), Julin Peng (jpeng49@wisc.edu), and Junyi Wei (jwei53@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Our goal was to attempt to model the decisions facing a farmer when choosing what to grow on a hobby farm.  Given a plot of land, the farmer needs to choose which crops to grow, which livestock to raise and which equipment to purchase.  Unlike large-scale commercial farms, hobby farms tend to grow a diverse mixture of crops and livestock.  There is evidence that hobby farms fail more quickly than other types of agriculture[1], for many reasons but financial difficulties appear to be the primary driver.  Optimizing the profitability of such an endeavor, while also considering the constraint of diverisity in attempt to achieve a more "idyllic" setting, increases the likelihood of success.

The complexity of this optimization problem is higher than it may seem at first glance.  On a large scale monocultural farm the problem is relatively simple, which crop has the maximum profit per acre?  On a hobby farm with a goal of diversity this is not the case.  Often, a decision about one crop or livestock is affected by another.  For example, if a farmer decides to grow a crop which can be used as feed for their animals they decrease the possible profit from that crop while also decreasing costs associated with that livestock.

Finding data with which to model this problem is not difficult.  In fact, the challenge is actually in sifting through the abundance of government and industry reports in order to track down the various values.  As our goal was attempting to create an accurate model of this problem, continuing to add more detail to bring it closer and closer to reality, government sources were identified and values for costs and returns pulled from their tables [2][3][4][5][6][7][8][9][10].  If the focus of the work had been on searching for a true optimal set of choices and maximized return these data sources would have been further validated and the values normalized to ensure cross-compatibility (e.g dates, locations, etc).

The following section will describe the iterative process used to model this problem.  Starting with a simplied "toy" model and then adding complexity in order to bring the model closer to the real world problem.  The model description is followed by [Julia] code using [JuMP] (Julia for Mathematical Optimization) in order to encode the model and find optimal results.  These results are then thoroughly described, with the final model being compared to previous, simplied iterations.  Finally, conclusions about the problem, model and process are presented.

***Citations:***

[1] Moyer, J. (2015). What nobody told me about small farming: I can’t make a living | Salon https://www.salon.com/2015/02/10/what_nobody_told_me_about_small_farming_i_cant_make_a_living/

[2] U. S. Department of Agriculture. (2017). Crop Values 2016 Summary. Washington, DC: Author.

[3] U. S. Department of Agriculture. (2018). Crop Production 2017 Summary. Washington, DC: Author.

[4] Manitoba Agriculture Farm Management. (2017). Guidelines for Estimating Crop Production Costs. Winnipeg, Manitoba: Author.

[5] Iowa State University Extension and Outreach. (2018). Estimated Returns to Farrow to Finish. Ames, IA: Author.

[6] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Yearling Steers. Ames, IA: Author.

[7] Iowa State University Extension and Outreach. (2018). Estimated Returns to Finishing Steer Calves. Ames, IA: Author.

[8] Iowa State University Extension and Outreach. (2018). Agriculture Decision Maker. Retrieved from https://www.extension.iastate.edu/agdm/ldcostsreturns.html

[9] UW-Madison College of Agriculture and Life Sciences. (2003). Large-scale pastured poultry farming in the U.S. Madison, WI: Author.

[10] Hamra, C.F. (2010). An Assessment of the Potential Profitability of Poultry Farms: A Broiler Farm Feasibility Case Study. Master's Thesis. University of Tennessee at Martin. Martin, TN.

   [Julia]: <https://julialang.org/>
   [JuMP]: <https://github.com/JuliaOpt/JuMP.jl>


## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$\begin{bmatrix}
      1 & 2 \\
       3 & 4
    \end{bmatrix}
    \begin{bmatrix} x \\ y \end{bmatrix} =
    \begin{bmatrix} 5 \\ 6 \end{bmatrix}$$

And here is an example of an optimization problem in standard form:
$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
    \text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
    & h_j(x) = 0 && j=1,\dots,r
    \end{aligned}$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
using DataFrames, CSV, NamedArrays
raw = CSV.read("crops.csv");
# turn Crops into an array
crops_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each crop;
info_crops = names(raw[2:end]);
# create a list of crops from the crops array
crops = crops_array[1:end,1];
# create a NamedArray that specifies the information of each crop
using NamedArrays
crops_matrix = crops_array[1:end,2:end] # rows are crops, columns are information of crops
crops_information_array = NamedArray(crops_matrix, (crops, info_crops), ("crops","info_crops"))

raw = CSV.read("LiveStock.csv");
# turn LiveStock into an array
lstocks_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each livestock;
info_lstocks = names(raw[2:end]);
# create a list of stocks from the livestocks array
lstocks = lstocks_array[1:end,1];
# create a NamedArray that specifies the information of each livestock
using NamedArrays
lstocks_matrix = lstocks_array[1:end,2:end] # rows are livestocks, columns are information of livestocks
lstocks_information_array = NamedArray(lstocks_matrix, (lstocks, info_lstocks), ("lstocks","info_lstocks"))

#the properties of our farm
tt_field=
tt_pasture=
tt_housing=

using JuMP, Clp
m = Model(solver=ClpSolver()) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crops[crops] >= 0)
@variable(m, lstocks[lstocks]>=0)

#constraints of field
@constraint(m, ttfield, sum(crops[i] for i in crops) <= tt_field )
#constraints of pasture
@constraint(m, ttpasture, sum(lstocks[i]*lstocks_information_array[i,:Pasture (acres/hd)] for i in lstocks) <= tt_pasture )
#constraints of housing area
@constraint(m, ttpasture, sum(lstocks[i]*lstocks_information_array[i,:EnclosedHousing(sq. ft./hd)] for i in lstocks) <= tt_housing )

# expression of the total return from crops
@expression(m, tot_return_crop, sum(crops[i]* crops_information_array[i,:Return ($/acre)]for i in crops) )
#expression of the total return from livestocks
@expression(m, tot_return_lstock, sum(lstocks[i]* lstocks_information_array[i,:Returns ($/hd)]for i in lstocks))

#objective: to maximize the return
@objective(m, Max, tot_return_crop+tot_return_lstocks)



The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

 Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are          | Cool  |
| ------------- |:-------------| -----:|
| col 3 is      |right-aligned |\$1600 |
|  colons       | align columns|  \$12 |
| zebra stripes |    are neat  |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.